In [1]:
import pandas as pd
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from collections import Counter
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D, Convolution1D
from keras.optimizers import Adadelta
from sklearn import metrics
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import json
import time
import datetime
import numpy as np

[nltk_data] Downloading package stopwords to /home/alice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-05-26 12:50:02.304794: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-26 12:50:02.304809: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Leitura e pré-processamento

In [2]:
def read_file_as_df(file_name):
    import pandas as pd
    import csv

    import sys
    import pandas as pd

    maxInt = sys.maxsize

    while True:
        # decrease the maxInt value by factor 10
        # as long as the OverflowError occurs.

        try:
            csv.field_size_limit(maxInt)
            break
        except OverflowError:
            maxInt = int(maxInt/10)

    file = []
    col = []

    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count==0:
                for r in row:
                    col.append(r)
                line_count+=1
            else:
                line = []
                for r in row:
                    line.append(r)
                file.append(line)
                line_count += 1


    df = pd.DataFrame(file, columns = col)
    return df

In [3]:
import string
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
import pandas as pd

[nltk_data] Downloading package stopwords to /home/alice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import re

def remove_between_dollars(text):
    return re.sub(r'\$.*?\$', '', text)

def remove_stopwords(text):
    all_words = text.split(" ")
    clean_text = [i for i in all_words if i not in stopwords and i!=""]
    return " ".join(clean_text)

def clean_text(x):
  return " ".join(str(x.translate(str.maketrans('', '', string.punctuation))).replace("#", "").split()).lower()

df = pd.read_csv("../data/b5-ready.csv")

df["Age"] = df["Age_Group"] - 1

df["Clean_text"] = df.Text.apply(lambda x: remove_between_dollars(x))
df["Clean_text"] = df.Clean_text.apply(lambda x: clean_text(x))
df["Clean_text"] = df.Clean_text.apply(lambda x: remove_stopwords(x))

X = df['Clean_text']
y = df["Age"]

In [5]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=958, stratify=y)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=958, stratify=y)

In [6]:
train_df.to_csv('../data/train.csv')
test_df.to_csv('../data/test.csv')

## Modelo baseado em dicionários

In [7]:
unique_words = list(set(" ".join(train_df['Clean_text'].to_list()).split()))
threshold = train_df['Clean_text'].shape[0]*0.01

word_counts = train_df['Clean_text'].str.split().explode().value_counts()
row_counts = pd.DataFrame({'word': word_counts.index, 'count': word_counts.values})
row_counts['count'] = row_counts['word'].apply(lambda word: train_df['Clean_text'].str.contains(word).sum())
selected_words = row_counts[row_counts['count']>threshold]['word'].to_list()

In [8]:
len(selected_words)

13869

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

text = train_df['Clean_text'].to_list()
vectorizer = TfidfVectorizer(vocabulary=selected_words)
train_matrix = vectorizer.fit_transform(text)
train_matrix = train_matrix.toarray()
vocab = vectorizer.get_feature_names_out()

In [10]:
len(vocab)

13869

In [11]:
tf_idf = pd.DataFrame(data=train_matrix, columns=selected_words)

In [12]:
tf_idf.head()

,pra,dia,vida,tudo,amor,vai,hoje,sempre,todos,bom,...,deixas,vitorio,fiam,culpas,poet,unte,mam,amaldiçoa,cross,bica
0,0.199915,0.109686,0.062982,0.026897,0.024089,0.046661,0.052744,0.055834,0.063762,0.032356,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.262431,0.047209,0.018072,0.041675,0.000000,0.066942,0.009080,0.019225,0.013721,0.032494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.011939,0.011910,0.025075,0.000000,0.028772,0.000000,0.025199,0.026675,0.076157,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.146931,0.107487,0.133726,0.253036,0.129834,0.081289,0.031012,0.109430,0.062484,0.031708,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.129215,0.000000,0.012336,0.050574,0.000000,0.012185,0.000000,0.013123,0.012489,0.012675,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
tf_idf.shape

(413, 13869)

In [14]:
X = tf_idf
y =  train_df['Age']

In [15]:
from sklearn.svm import SVC

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'tol': [1e-4, 1e-3, 1e-2],  # Tolerance for stopping criteria
    'max_iter': [1000, 5000, 10000, -1] # Maximum number of iterations
}

In [18]:
svc = SVC(kernel='linear', decision_function_shape='ovr')
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, scoring='f1_macro', verbose=2)

# Fit the model to the training data
grid_search.fit(X, y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END ...................C=0.1, max_iter=1000, tol=0.0001; total time=   1.4s
[CV] END ...................C=0.1, max_iter=1000, tol=0.0001; total time=   1.3s
[CV] END ...................C=0.1, max_iter=1000, tol=0.0001; total time=   1.2s
[CV] END ...................C=0.1, max_iter=1000, tol=0.0001; total time=   1.2s
[CV] END ...................C=0.1, max_iter=1000, tol=0.0001; total time=   1.3s
[CV] END ....................C=0.1, max_iter=1000, tol=0.001; total time=   1.4s
[CV] END ....................C=0.1, max_iter=1000, tol=0.001; total time=   1.3s
[CV] END ....................C=0.1, max_iter=1000, tol=0.001; total time=   1.3s
[CV] END ....................C=0.1, max_iter=1000, tol=0.001; total time=   1.3s
[CV] END ....................C=0.1, max_iter=1000, tol=0.001; total time=   1.2s
[CV] END .....................C=0.1, max_iter=1000, tol=0.01; total time=   1.3s
[CV] END .....................C=0.1, max_iter=1

GridSearchCV(cv=5, estimator=SVC(kernel='linear'),
             param_grid={'C': [0.1, 1, 10, 100],
                         'max_iter': [1000, 5000, 10000, -1],
                         'tol': [0.0001, 0.001, 0.01]},
             scoring='f1_macro', verbose=2)

In [19]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation accuracy: {best_score}")

Best parameters: {'C': 1, 'max_iter': 1000, 'tol': 0.0001}
Best cross-validation accuracy: 0.5523587436855141


In [20]:
svc = SVC(kernel='linear', decision_function_shape='ovr', C=1, gamma='scale', max_iter=1000, tol=0.0001)
svc.fit(X, y)

SVC(C=1, kernel='linear', max_iter=1000, tol=0.0001)

In [21]:
df_coef = pd.DataFrame(data=svc.coef_, columns=tf_idf.columns)

In [22]:
df_coef = df_coef.T

In [23]:
df_coef.head()

,0,1,2
pra,0.087496,0.351631,0.343733
dia,-0.361199,-0.801266,-0.571707
vida,0.423922,0.219963,-0.130838
tudo,0.491163,0.671665,0.447189
amor,0.280907,0.785810,0.625140


In [24]:
def select_abs(col, perc): # mesma quantidade de palavras positivas e negativas
  n = int(len(df_coef) * perc)

  df_coef_1 = df_coef.copy()
  df_coef_1 = df_coef_1[[col]]

  df_coef_1['abs_values'] = df_coef_1[col].abs()

  sorted_df = df_coef_1.sort_values(by='abs_values', ascending=False)

  return sorted_df.head(n)

def select_(col, perc): # mesma quantidade de palavras positivas e negativas
  df_coef_1 = df_coef.copy()
  df_coef_1 = df_coef_1[[col]]

  str_col = str(col)

  df_coef_1.rename(columns={col: str_col}, inplace=True)

  df_coef_1_pos = df_coef_1[df_coef_1[str_col]>0]
  df_coef_1_neg= df_coef_1[df_coef_1[str_col]<0]
  df_coef_1_zero = df_coef_1[df_coef_1[str_col]==0]

  df_coef_1_pos = df_coef_1_pos.sort_values(by=str_col, ascending=True)
  df_coef_1_neg = df_coef_1_neg.sort_values(by=str_col, key=lambda x: abs(x))

  pos_perc = df_coef_1_pos.head(int(len(df_coef_1_pos) * perc))
  neg_perc = df_coef_1_neg.head(int(len(df_coef_1_neg) * perc))

  indices_to_drop = pd.concat([pos_perc, neg_perc, df_coef_1_zero]).index
  print(df_coef_1_pos.shape, df_coef_1_neg.shape, pos_perc.shape, neg_perc.shape, df_coef_1_zero.shape, len(indices_to_drop))

  df_filtered = df_coef_1.drop(indices_to_drop)

  return df_filtered.sort_values(by=str_col, ascending=False)

coef1 = select_abs(0, 1)
coef2 = select_abs(1, 1)
coef3 = select_abs(2, 1)

In [ ]:
# coef1.sort_values(by=0, ascending=False)[[0]].to_csv('../data/dict1.csv')
# coef2.sort_values(by=1, ascending=False)[[1]].to_csv('../data/dict2.csv')
# coef3.sort_values(by=2, ascending=False)[[2]].to_csv('../data/dict3.csv')

In [25]:
coef1.head(10), coef2.head(10), coef3.head(10) # ver se as palavras estão realmente fazendo sentido: positivo -> aparece muito, negativo -> parace pouco e muito em outras classes

(                0  abs_values
 amo      1.534984    1.534984
 cunhada -1.200581    1.200581
 the     -1.154011    1.154011
 mãe     -1.119207    1.119207
 coronel -1.030501    1.030501
 true    -0.941845    0.941845
 lindo   -0.919721    0.919721
 galera  -0.899805    0.899805
 cartão  -0.872849    0.872849
 copa    -0.828210    0.828210,
                 1  abs_values
 amo      1.321390    1.321390
 you      1.174515    1.174515
 olha    -1.173966    1.173966
 usp      1.003721    1.003721
 ae       0.822172    0.822172
 gente    0.818730    0.818730
 família -0.818659    0.818659
 dia     -0.801266    0.801266
 melhor   0.798625    0.798625
 amanhã   0.795486    0.795486,
                      2  abs_values
 usp           1.207524    1.207524
 of            0.899497    0.899497
 xd            0.867477    0.867477
 to            0.836178    0.836178
 risos        -0.823191    0.823191
 participando  0.816212    0.816212
 you           0.812044    0.812044
 erro          0.801332    0

In [ ]:
coef1['0']['amei'], coef2['1']['amei'], coef3['2']['amei']

(0.8168308415641511, 0.3213758181515176, -0.2848448681909091)

In [ ]:
coef1['0']['cunhada'], coef2['1']['cunhada'], coef3['2']['cunhada']

(-1.200580730966745, -0.4201863386820712, 0.6289896418172819)

In [ ]:
df_coef.shape, coef1.shape

((13869, 3), (6561, 1))

In [26]:
train_df['glex1']=0
train_df['glex2']=0
train_df['glex3']=0

In [27]:
svc.intercept_

array([ 0.23662019,  0.03069715, -0.143966  ])

In [28]:
len(coef1.index.tolist())

13869

In [29]:
train_df.reset_index(inplace=True)

In [30]:
train_df

,index,Unnamed: 0,User_ID,age,Text,Age_Group,Text_TK,Age,Clean_text,glex1,glex2,glex3
0,231,547,628,24.0,"por $NAME$ de . # "" https : www . $NAME$ . co...",2,"['por', '$', 'NAME', '$', 'de', '.', '#', '``'...",1,https www watch v k liberado contar ás horas o...,0,0,0
1,17,22,40,40.0,Dia triste em que eu tenho que me preocupar aq...,3,"['Dia', 'triste', 'em', 'que', 'eu', 'tenho', ...",2,dia triste preocupar aqui merdas fcc quer faze...,0,0,0
2,463,966,1090,31.0,ditadura temer # $LAUGH$ # Um político sucat...,3,"['ditadura', 'temer', '#', '$', 'LAUGH', '$', ...",2,ditadura temer político sucateado softskinlase...,0,0,0
3,136,383,442,19.0,""" Sorte tenho eu , de ter você aqui $NAME$ "" ...",1,"['``', 'Sorte', 'tenho', 'eu', ',', 'de', 'ter...",0,sorte ter aqui dia setembro virou especiais an...,0,0,0
4,425,911,1029,18.0,""" Eu espero que ninguém tente justificar mais ...",1,"['``', 'Eu', 'espero', 'que', 'ninguém', 'tent...",0,espero ninguém tente justificar nada comentári...,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
408,360,797,903,25.0,Que tal ? $LAUGH$ # Nada mal . . $LAUGH$ # É...,2,"['Que', 'tal', '?', '$', 'LAUGH', '$', '#', 'N...",1,tal nada mal vida quer vida quer capote rei qu...,0,0,0
409,322,722,822,19.0,< $NUMBER$ # Corre lá ! # $EMOTION*$ < $NUMB...,1,"['<', '$', 'NUMBER', '$', '#', 'Corre', 'lá', ...",0,corre lá prayforparis veste vestidinho feito m...,0,0,0
410,305,684,779,23.0,""" Depois de um ano , finalmente posso descansa...",2,"['``', 'Depois', 'de', 'um', 'ano', ',', 'fina...",1,ano finalmente posso descansar paz ton acontec...,0,0,0
411,333,737,837,18.0,Temos uma vida inteira pela frente $NUMBER$ An...,1,"['Temos', 'uma', 'vida', 'inteira', 'pela', 'f...",0,vida inteira frente ano meses cool the gracios...,0,0,0


In [32]:
# dict 1
for index, row in train_df.iterrows():
    sum = 0
    text = row["Clean_text"].split()
    for w in text:
      if w in coef1.index.tolist():
        sum += coef1[0][w]*tf_idf[w][index]
    sum = sum + svc.intercept_[0]
    train_df[f'glex1'][index] = sum

/tmp/ipykernel_2702/374887734.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[f'glex1'][index] = sum


In [33]:
# dict 1
for index, row in train_df.iterrows():
    sum = 0
    text = row["Clean_text"].split()
    for w in text:
      if w in coef2.index.tolist():
        sum += coef2[1][w]*tf_idf[w][index]
    sum = sum + svc.intercept_[1]
    train_df[f'glex2'][index] = sum

/tmp/ipykernel_2702/3689925062.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[f'glex2'][index] = sum


In [34]:
# dict 1
for index, row in train_df.iterrows():
    sum = 0
    text = row["Clean_text"].split()
    for w in text:
      if w in coef3.index.tolist():
        sum += coef3[2][w]*tf_idf[w][index]
    sum = sum + svc.intercept_[2]
    train_df[f'glex3'][index] = sum

/tmp/ipykernel_2702/914802386.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[f'glex3'][index] = sum


In [35]:
train_df.groupby(['Age_Group'])[['glex1', 'glex2', 'glex3']].describe().T

Age_Group             1           2           3
glex1 count  145.000000  151.000000  117.000000
      mean     4.730472   -6.739027   -1.723705
      std      5.840970   37.602047    7.299686
      min     -4.376449 -454.532836  -67.387710
      25%      1.395280   -5.197720   -2.939187
      50%      2.608160   -2.945432   -1.034262
      75%      5.911522   -1.236714    0.108804
      max     31.095782   42.987714   19.951026
glex2 count  145.000000  151.000000  117.000000
      mean     7.101108    2.940649   -4.729961
      std      7.278179   15.837429   13.246202
      min     -9.749402 -158.918588 -133.045171
      25%      2.310134    0.192533   -4.949022
      50%      4.474745    1.785529   -2.542779
      75%      9.185887    4.148670   -1.377447
      max     40.173765   46.519221   10.253153
glex3 count  145.000000  151.000000  117.000000
      mean     4.152673    9.133123   -3.788808
      std      5.718207   20.742842   12.408786
      min    -15.830419   -1.682541 -110.399490
      25%      0.808243    2.684311   -4.117260
      50%      2.515870    4.826517   -2.165204
      75%      6.463041    8.973259   -0.957995
      max     29.219679  238.354855   41.614098

In [36]:
train_df['c'] = 0

for index, row in train_df.iterrows():
  if row['glex1'] > 0 and row['glex2'] > 0 and row['glex3'] > 0:
    train_df['c'][index] = 1
  elif row['glex1'] < 0 and row['glex2'] > 0 and row['glex3'] > 0:
    train_df['c'][index] = 2
  elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < 0:
    train_df['c'][index] = 3

/tmp/ipykernel_2702/2201815617.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['c'][index] = 2
/tmp/ipykernel_2702/2201815617.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['c'][index] = 3
/tmp/ipykernel_2702/2201815617.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['c'][index] = 1


In [37]:
train_df['Age_Group'] = train_df['Age_Group'].astype('int64')

In [38]:
# print(f'Instâncias totais: {train_df.shape[0]}')
# print(f'Instâncias classificadas: {train_df[train_df.c!=0].shape[0]}')
# print(f'Porcentagem classificadas: {train_df[train_df.c!=0].shape[0]/train_df.shape[0]}')
# print(f'Instâncias classificadas corretamente: {train_df[(train_df.c!=0) & (train_df.c==train_df.Age_Group)].shape[0]}')
# print(f'Porcentagem classificadas corretamente: {train_df[(train_df.c!=0) & (train_df.c==train_df.Age_Group)].shape[0]/train_df[train_df.c!=0].shape[0]}')
# print(f'Porcentagem classificadas corretamente das totais: {train_df[(train_df.c!=0) & (train_df.c==train_df.Age_Group)].shape[0]/train_df.shape[0]}')

In [39]:
# print(f'Porcentagem classificadas corretamente da classe 1: {train_df[(train_df.c!=0) & (train_df.c==train_df.Age_Group) & (train_df.Age_Group==1)].shape[0]/train_df[(train_df.c!=0)&(train_df.Age_Group==1)].shape[0]}')
# print(f'Porcentagem classificadas corretamente da classe 2: {train_df[(train_df.c!=0) & (train_df.c==train_df.Age_Group) & (train_df.Age_Group==2)].shape[0]/train_df[(train_df.c!=0)&(train_df.Age_Group==2)].shape[0]}')
# print(f'Porcentagem classificadas corretamente da classe 3: {train_df[(train_df.c!=0) & (train_df.c==train_df.Age_Group) & (train_df.Age_Group==3)].shape[0]/train_df[(train_df.c!=0)&(train_df.Age_Group==3)].shape[0]}')

In [40]:
test_matrix = vectorizer.transform(test_df["Clean_text"].to_list())
test_matrix = test_matrix.toarray()

tf_idf_test = pd.DataFrame(data=test_matrix, columns=selected_words)

In [41]:
test_df.reset_index(inplace=True)

In [42]:
test_df['glex1']=0
test_df['glex2']=0
test_df['glex3']=0

In [43]:
# dict 1
for index, row in test_df.iterrows():
    sum = 0
    text = row["Clean_text"].split()
    for w in text:
      if w in coef1.index.tolist():
        sum += coef1[0][w]*tf_idf_test[w][index]
    sum = sum + svc.intercept_[0]
    test_df[f'glex1'][index] = sum

/tmp/ipykernel_2702/2181702926.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[f'glex1'][index] = sum


In [44]:
# dict 1
for index, row in test_df.iterrows():
    sum = 0
    text = row["Clean_text"].split()
    for w in text:
      if w in coef2.index.tolist():
        sum += coef2[1][w]*tf_idf_test[w][index]
    sum = sum + svc.intercept_[1]
    test_df[f'glex2'][index] = sum

/tmp/ipykernel_2702/3566863503.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[f'glex2'][index] = sum


In [45]:
# dict 1
for index, row in test_df.iterrows():
    sum = 0
    text = row["Clean_text"].split()
    for w in text:
      if w in coef3.index.tolist():
        sum += coef3[2][w]*tf_idf_test[w][index]
    sum = sum + svc.intercept_[2]
    test_df[f'glex3'][index] = sum

/tmp/ipykernel_2702/2848492172.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[f'glex3'][index] = sum


In [46]:
test_df.groupby(['Age_Group'])[['glex1', 'glex2', 'glex3']].describe().T

Age_Group            1          2          3
glex1 count  37.000000  38.000000  29.000000
      mean    1.797664  -1.202727  -0.124507
      std     3.568597  11.112098   3.483563
      min    -3.284092 -53.709392  -4.372061
      25%    -0.133737  -1.836902  -2.146671
      50%     0.439253  -0.748632  -0.207805
      75%     2.648331   0.846483   0.419674
      max    14.846039  30.983051  12.139912
glex2 count  37.000000  38.000000  29.000000
      mean    5.366819   4.849448   0.116724
      std     5.238562  10.933698   4.448097
      min    -1.573880  -5.282855 -13.797514
      25%     0.555623   0.092453  -1.420358
      50%     4.035966   2.061635  -0.113923
      75%     9.145498   5.040851   0.832258
      max    18.032934  46.991266  13.212151
glex3 count  37.000000  38.000000  29.000000
      mean    4.992779   6.292395   0.251084
      std     5.030369  14.660386   3.994880
      min    -2.920832  -5.275918 -12.875705
      25%     1.054277   1.159473  -0.615939
      50%     4.085012   2.380234   0.244840
      75%     9.372102   5.703360   2.139951
      max    20.599419  87.206293   8.361132

In [47]:
test_df['Age_Group'] = test_df['Age_Group'].astype('int64')

In [49]:
test_df['c'] = 0

for index, row in test_df.iterrows():
  if row['glex1'] > 0 and row['glex2'] > 0 and row['glex3'] > 0: # delimitaria só pelo glex da regra
    test_df['c'][index] = 1
  elif row['glex1'] < 0 and row['glex2'] > 0 and row['glex3'] > 0:
    test_df['c'][index] = 2
  elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < 0:
    test_df['c'][index] = 3

test_df_dict = test_df[test_df['c'] != 0]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['c'].to_list(), test_df_dict['Age_Group'].to_list(), average='macro')}")

/tmp/ipykernel_2702/783782555.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c'][index] = 2
/tmp/ipykernel_2702/783782555.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c'][index] = 1
/tmp/ipykernel_2702/783782555.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c'][index] = 3


F1 (dict): 0.5098412698412699


In [51]:
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.c!=0].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.c!=0].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.c!=0) & (test_df.c==test_df.Age_Group)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.c!=0) & (test_df.c==test_df.Age_Group)].shape[0]/test_df[test_df.c!=0].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.c!=0) & (test_df.c==test_df.Age_Group)].shape[0]/test_df.shape[0]}')

Instâncias totais: 104
Instâncias classificadas: 84
Porcentagem classificadas: 0.8076923076923077
Instâncias classificadas corretamente: 44
Porcentagem classificadas corretamente: 0.5238095238095238
Porcentagem classificadas corretamente das totais: 0.4230769230769231


In [52]:
print(f'Porcentagem classificadas corretamente da classe 1: {test_df[(test_df.c!=0) & (test_df.c==test_df.Age_Group) & (test_df.Age_Group==1)].shape[0]/test_df[(test_df.c!=0)&(test_df.Age_Group==1)].shape[0]}')
print(f'Porcentagem classificadas corretamente da classe 2: {test_df[(test_df.c!=0) & (test_df.c==test_df.Age_Group) & (test_df.Age_Group==2)].shape[0]/test_df[(test_df.c!=0)&(test_df.Age_Group==2)].shape[0]}')
print(f'Porcentagem classificadas corretamente da classe 3: {test_df[(test_df.c!=0) & (test_df.c==test_df.Age_Group) & (test_df.Age_Group==3)].shape[0]/test_df[(test_df.c!=0)&(test_df.Age_Group==3)].shape[0]}')

Porcentagem classificadas corretamente da classe 1: 0.6470588235294118
Porcentagem classificadas corretamente da classe 2: 0.4838709677419355
Porcentagem classificadas corretamente da classe 3: 0.3684210526315789


In [53]:
df_stat = train_df.groupby(['Age_Group'])[['glex1', 'glex2', 'glex3']].describe().T

In [54]:
test_df['c50'] = 0

for index, row in test_df.iterrows():
  if row['glex1'] > df_stat[1]['glex1']['50%'] and row['glex2'] > 0 and row['glex3'] > 0: # delimitaria só pelo glex da regra
    test_df['c50'][index] = 1
  elif row['glex1'] < 0 and row['glex2'] > df_stat[2]['glex2']['50%'] and row['glex3'] > 0:
    test_df['c50'][index] = 2
  elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < df_stat[3]['glex3']['50%']:
    test_df['c50'][index] = 3

test_df_dict = test_df[test_df['c50'] != 0]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['c50'].to_list(), test_df_dict['Age_Group'].to_list(), average='macro')}")
print("")
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.c50!=0].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.c50!=0].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.c50!=0) & (test_df.c50==test_df.Age_Group)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.c50!=0) & (test_df.c50==test_df.Age_Group)].shape[0]/test_df[test_df.c50!=0].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.c50!=0) & (test_df.c50==test_df.Age_Group)].shape[0]/test_df.shape[0]}')

/tmp/ipykernel_2702/553891255.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c50'][index] = 2
/tmp/ipykernel_2702/553891255.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c50'][index] = 3
/tmp/ipykernel_2702/553891255.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c50'][index] = 1


F1 (dict): 0.5113095238095239

Instâncias totais: 104
Instâncias classificadas: 35
Porcentagem classificadas: 0.33653846153846156
Instâncias classificadas corretamente: 19
Porcentagem classificadas corretamente: 0.5428571428571428
Porcentagem classificadas corretamente das totais: 0.18269230769230768


In [55]:
test_df['c5025'] = 0

for index, row in test_df.iterrows():
  if row['glex1'] > df_stat[1]['glex1']['50%'] and row['glex2'] > df_stat[1]['glex2']['25%'] and row['glex3'] > df_stat[1]['glex3']['25%']: # delimitaria só pelo glex da regra
    test_df['c5025'][index] = 1
  elif row['glex1'] < df_stat[2]['glex1']['25%'] and row['glex2'] > df_stat[2]['glex2']['50%'] and row['glex3'] > df_stat[2]['glex3']['25%']:
    test_df['c5025'][index] = 2
  elif row['glex1'] < df_stat[3]['glex1']['25%'] and row['glex2'] < df_stat[3]['glex2']['25%'] and row['glex3'] < df_stat[3]['glex3']['50%']:
    test_df['c5025'][index] = 3

test_df_dict = test_df[test_df['c5025'] != 0]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['c5025'].to_list(), test_df_dict['Age_Group'].to_list(), average='macro')}")
print("")
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.c5025!=0].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.c5025!=0].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.c5025!=0) & (test_df.c5025==test_df.Age_Group)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.c5025!=0) & (test_df.c5025==test_df.Age_Group)].shape[0]/test_df[test_df.c5025!=0].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.c5025!=0) & (test_df.c5025==test_df.Age_Group)].shape[0]/test_df.shape[0]}')

/tmp/ipykernel_2702/102490375.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c5025'][index] = 2
/tmp/ipykernel_2702/102490375.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c5025'][index] = 3
/tmp/ipykernel_2702/102490375.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c5025'][index] = 1


F1 (dict): 0.6166666666666666

Instâncias totais: 104
Instâncias classificadas: 19
Porcentagem classificadas: 0.18269230769230768
Instâncias classificadas corretamente: 13
Porcentagem classificadas corretamente: 0.6842105263157895
Porcentagem classificadas corretamente das totais: 0.125


In [56]:
test_df['cmean'] = 0

for index, row in test_df.iterrows():
  if row['glex1'] > df_stat[1]['glex1']['mean'] and row['glex2'] > 0 and row['glex3'] > 0: # delimitaria só pelo glex da regra
    test_df['cmean'][index] = 1
  elif row['glex1'] < 0 and row['glex2'] > df_stat[2]['glex2']['mean'] and row['glex3'] > 0:
    test_df['cmean'][index] = 2
  elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < df_stat[3]['glex3']['mean']:
    test_df['cmean'][index] = 3

test_df_dict = test_df[test_df['cmean'] != 0]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['cmean'].to_list(), test_df_dict['Age_Group'].to_list(), average='macro')}")
print("")
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.cmean!=0].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.cmean!=0].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.cmean!=0) & (test_df.cmean==test_df.Age_Group)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.cmean!=0) & (test_df.cmean==test_df.Age_Group)].shape[0]/test_df[test_df.cmean!=0].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.cmean!=0) & (test_df.cmean==test_df.Age_Group)].shape[0]/test_df.shape[0]}')

/tmp/ipykernel_2702/1224727425.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cmean'][index] = 2
/tmp/ipykernel_2702/1224727425.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cmean'][index] = 3
/tmp/ipykernel_2702/1224727425.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cmean'][index] = 1


F1 (dict): 0.5462074157726332

Instâncias totais: 104
Instâncias classificadas: 26
Porcentagem classificadas: 0.25
Instâncias classificadas corretamente: 14
Porcentagem classificadas corretamente: 0.5384615384615384
Porcentagem classificadas corretamente das totais: 0.1346153846153846


In [57]:
test_df['c75'] = 0

for index, row in test_df.iterrows():
  if row['glex1'] > df_stat[1]['glex1']['75%'] and row['glex2'] > 0 and row['glex3'] > 0: # delimitaria só pelo glex da regra
    test_df['c75'][index] = 1
  elif row['glex1'] < 0 and row['glex2'] > df_stat[2]['glex2']['75%'] and row['glex3'] > 0:
    test_df['c75'][index] = 2
  elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < df_stat[3]['glex3']['75%']:
    test_df['c75'][index] = 3

test_df_dict = test_df[test_df['c75'] != 0]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['c75'].to_list(), test_df_dict['Age_Group'].to_list(), average='macro')}")
print("")
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.c75!=0].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.c75!=0].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.c75!=0) & (test_df.c75==test_df.Age_Group)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.c75!=0) & (test_df.c75==test_df.Age_Group)].shape[0]/test_df[test_df.c75!=0].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.c75!=0) & (test_df.c75==test_df.Age_Group)].shape[0]/test_df.shape[0]}')

/tmp/ipykernel_2702/2897540198.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c75'][index] = 2
/tmp/ipykernel_2702/2897540198.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c75'][index] = 3
/tmp/ipykernel_2702/2897540198.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c75'][index] = 1


F1 (dict): 0.5622710622710622

Instâncias totais: 104
Instâncias classificadas: 27
Porcentagem classificadas: 0.25961538461538464
Instâncias classificadas corretamente: 15
Porcentagem classificadas corretamente: 0.5555555555555556
Porcentagem classificadas corretamente das totais: 0.14423076923076922


In [58]:
test_df['c7525'] = 0

for index, row in test_df.iterrows():
  if row['glex1'] > df_stat[1]['glex1']['75%'] and row['glex2'] >  df_stat[1]['glex2']['25%'] and row['glex3'] > df_stat[1]['glex3']['25%']: # delimitaria só pelo glex da regra
    test_df['c7525'][index] = 1
  elif row['glex1'] < df_stat[2]['glex1']['25%'] and row['glex2'] > df_stat[2]['glex2']['75%'] and row['glex3'] > df_stat[2]['glex3']['25%']:
    test_df['c7525'][index] = 2
  elif row['glex1'] < df_stat[3]['glex1']['25%'] and row['glex2'] < df_stat[3]['glex2']['25%'] and row['glex3'] < df_stat[3]['glex3']['75%']:
    test_df['c7525'][index] = 3

test_df_dict = test_df[test_df['c7525'] != 0]
print(f"F1 (dict): {metrics.f1_score(test_df_dict['c7525'].to_list(), test_df_dict['Age_Group'].to_list(), average='macro')}")
print("")
print(f'Instâncias totais: {test_df.shape[0]}')
print(f'Instâncias classificadas: {test_df[test_df.c7525!=0].shape[0]}')
print(f'Porcentagem classificadas: {test_df[test_df.c7525!=0].shape[0]/test_df.shape[0]}')
print(f'Instâncias classificadas corretamente: {test_df[(test_df.c7525!=0) & (test_df.c7525==test_df.Age_Group)].shape[0]}')
print(f'Porcentagem classificadas corretamente: {test_df[(test_df.c7525!=0) & (test_df.c7525==test_df.Age_Group)].shape[0]/test_df[test_df.c7525!=0].shape[0]}')
print(f'Porcentagem classificadas corretamente das totais: {test_df[(test_df.c7525!=0) & (test_df.c7525==test_df.Age_Group)].shape[0]/test_df.shape[0]}')

F1 (dict): 0.861111111111111

Instâncias totais: 104
Instâncias classificadas: 11
Porcentagem classificadas: 0.10576923076923077
Instâncias classificadas corretamente: 9
Porcentagem classificadas corretamente: 0.8181818181818182
Porcentagem classificadas corretamente das totais: 0.08653846153846154


/tmp/ipykernel_2702/2403513126.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c7525'][index] = 2
/tmp/ipykernel_2702/2403513126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c7525'][index] = 3
/tmp/ipykernel_2702/2403513126.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['c7525'][index] = 1


## Cascata

In [59]:
test_df_class = test_df[test_df.c7525!=0]

In [60]:
test_df_no_class = test_df[test_df.c7525==0]

In [61]:
test_df_class[['User_ID', 'Clean_text', 'Age_Group', 'Age']].head()

,User_ID,Clean_text,Age_Group,Age
3,276,b melo sabe programar manja ruby on rails venh...,2,1
4,598,companheiras banca ciência prezados amigos ami...,3,2
14,1122,morri dia mãe linda hoje sempre dia agradecer ...,1,0
22,972,olha machado each resiste usp greve aí coração...,1,0
27,1119,alive horas volta rumo qualquer esforço caber ...,2,1


In [62]:
test_df_class.to_csv('../data/dict-class.csv', encoding='latin-1')

In [63]:
test_df_no_class.to_csv('../data/no-class.csv', encoding='latin-1')

In [64]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adadelta
from keras.callbacks import EarlyStopping

def create_model_age(filters=[100], kernel_size=[50], strides=[100],
                     dropout_rate=0.5, pool_size=[5], dense_units=100, max_len=1000):

    model = Sequential()

    # First convolutional layer with deterministic initialization
    model.add(Conv1D(filters=filters[0],
                     kernel_size=kernel_size[0],
                     strides=strides[0],
                     activation='relu',
                     input_shape=(max_len, 1),
                     kernel_initializer=GlorotUniform(seed=42)))

    # MaxPooling and activation layers
    for i in range(len(pool_size)):
        model.add(MaxPooling1D(pool_size=pool_size[i], strides=1))
        model.add(Activation('relu'))

    model.add(Flatten())

    # Dropout layer without a seed (as it's deprecated for Dropout in TensorFlow 2.x)
    if dropout_rate is not None:
        model.add(Dropout(dropout_rate))

    # Dense layers with fixed initializers
    model.add(Dense(units=dense_units, activation='relu', kernel_initializer=GlorotUniform(seed=42)))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=GlorotUniform(seed=42)))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adadelta(learning_rate=1),
                  metrics=['accuracy'])

    return model


In [65]:
import math

In [66]:
import tensorflow as tf
import numpy as np
import random
import os

# Set the random seeds for reproducibility
seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

tf.config.experimental.enable_op_determinism()

In [68]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=958, stratify=y)
#df_train, df_test = train_test_split(df, test_size=0.2, random_state=958, stratify=y)

X_train = train_df['Clean_text']
y_train = train_df['Age'].to_numpy()

X_test = test_df['Clean_text']
y_test = test_df['Age'].to_numpy()

num_words = []
for text in X_train.tolist():
    num_words.append(len(text.split()))
mean_w = sum(num_words)//len(num_words)

train_texts = X_train.tolist()
test_texts = X_test.tolist()
tfidfvec = TfidfVectorizer(max_features = mean_w, max_df=0.9)
tfidfvec.fit(train_texts)
tfidf_train = tfidfvec.transform(train_texts).toarray()
tfidf_test = tfidfvec.transform(test_texts).toarray()

X_train_resampled, y_train_resampled = oversampling(tfidf_train, y_train)

X_train = X_train_resampled.reshape(X_train_resampled.shape[0],X_train_resampled.shape[1],1)
X_test = tfidf_test.reshape(tfidf_test.shape[0],tfidf_test.shape[1],1)
y_train= keras.utils.to_categorical(y_train_resampled,num_classes=3)
y_test = keras.utils.to_categorical(y_test,num_classes=3)

size = X_test.shape[1]

f1 = []

model = create_model_age(filters=[60], kernel_size=[1], strides=[1], dropout_rate=0.1, pool_size=[4], dense_units = 512, max_len = X_test.shape[1])
callback = [
            EarlyStopping(patience=5, monitor='val_accuracy', min_delta=0.1, mode='max', restore_best_weights=True),
        ]

model.fit(X_train,y_train,validation_data=(X_test,y_test), batch_size=32, epochs=10, callbacks=callback)

y_pred = model.predict(
        X_test
)

import numpy as np

y_pred_list = [np.argmax(x, axis=-1) for x in y_pred]
y_test_list =[np.argmax(x, axis=-1) for x in y_test]

print(metrics.f1_score(y_test_list, y_pred_list, average='macro'))

TypeError: 'numpy.float64' object is not callable

In [69]:
print(sum)

0.8360123490248782


In [ ]:
metrics.f1_score(y_test_tot, y_pred_tot, average='macro')

0.5961538461538461